# ANOVA Hintergründe 

Im letzten Notebook haben wir uns angesehen, wie eine ANOVA durchzuführen ist. Allerdings ist die dahinterstehende Mathematik nicht trivial. Deswegen und um es selbst nochmal zu wiederholen möchte ich mich mit euch in dem Notebook hier einmal mit der Statistik dahinter und den ganzen Werten beschäftigen. In der Hoffnung, dass das ganze nicht mehr ganz so fremd wirkt.

Im letzten Teil habe ich euch schon mal diese Gleichung gezeigt:

$$ F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Within\:group\:variability} $$

Am Ende des Notebooks steht wieder die Formel für den F-Wert, doch dann sollte das ganze hoffentlich sehr viel mehr mit Inhalt für euch gefüllt sein.

In [ ]:
import pandas as pd
import pingouin as pg

penguins = pd.read_csv("./penguins_classification.csv")

anova_results = pg.anova(
    data=penguins, dv="Culmen Length (mm)", between="Species", detailed=True
)

anova_results = anova_results.set_index("Source")
anova_results

## Sum of Squares

Nun möchte ich mir nochmal genauer mit euch ansehen, wie diese verschiedenen Varianzen berechnet werden, um ein besseres Gefühl für die Anwendung einer ANOVA zu bekommen, damit eine ANOVA nicht nur Statistik-Voodoo ist und ihr wisst, wo die ganzen Outputs einer ANOVA herkommen. Als ersten Schritt erstellen wir hierfür sogenannte Sums of Square (SS) die sind euch auch schon in der Output-Tabelle der ANOVA begegnet.

$$SS = \sum (x_i-M_{gesamt})^2$$

D.h. von jeder Culmen Lenght (mm) wurde für die Berechnung der SS einmal der Gesamtmittelwert abgezogen und der erhaltene Wert wurde quadriert. Die Formel erinnert euch vielleicht an die allgemeine Formel für Varianz.

$$\sigma^2 = \frac{\sum (x_i - M)^2}{N}$$

Für die Varianz würde man also eigentlich nur noch durch die Stichprobengröße teilen.

In [ ]:
import numpy as np

x = penguins["Culmen Length (mm)"].to_numpy()
M_gesamt = np.mean(x)

# ** ist die Schreibweise für Potenzen in Python, also 2 ** 2 = 4, und 3 ** 2 = 9, usw.
# np.sum nimmt die Summe

SS = np.sum((x - M_gesamt) ** 2)

print(
    f"Die SS gesamt ausgerechnet: {SS:.2f}.\n"
    f"SS gesamt aus der ANOVA-Tabelle: {anova_results['SS']['Species'] + anova_results['SS']['Within']:.2f}."
)

### $SS_{within}$

Die SS können wir auch für jede Gruppe der Pinguine errechnen. Damit das ganze etwas einfacher ist, schreiben wir hierfür eine Funktion.

Funktionen haben in Python den folgenden Aufbau wobei in <> immer Teile stehen, wo ihr Namen dafür frei wählen könnt.

```
def <Name der Funktion>(<Input1>, <Input2>, ..., <InputN>):
    
    Irgendwelche Operationen, die die Inputs nutzen und einen <Output> produzieren

    return <Output>

```

In [ ]:
def SumOfSquares(input_list):
    """
    input_list - Liste mit Daten für die, die Sum of Squares berechnet werden soll.
    """
    x = (
        input_list.to_numpy()
    )  # .to_numpy macht aus dem Pandas Dataframe/Series etwas womit numpy gut rechnen kann
    Mx = np.mean(x)

    return np.sum((x - Mx) ** 2)

In [ ]:
# Für unsere Funktion müssen wir immer erst filtern, dass 1. nur die Pinguine einer Spezies übrig sind und
# 2. nur die Werte für die Schnabelgröße
SS_Adelie = SumOfSquares(
    penguins[penguins["Species"] == "Adelie"]["Culmen Length (mm)"]
)
SS_Gentoo = SumOfSquares(
    penguins[penguins["Species"] == "Gentoo"]["Culmen Length (mm)"]
)
SS_Chinstrap = SumOfSquares(
    penguins[penguins["Species"] == "Chinstrap"]["Culmen Length (mm)"]
)

print(
    f"Summe der Varianzen innerhalb der Gruppen: {SS_Adelie + SS_Gentoo + SS_Chinstrap:.2f}.\n"
    f"Within Group Varianz aus der ANOVA-Tabelle: {anova_results['SS']['Within']:.2f}."
)

Q: Mache dir nochmal bewusst, wie oben aus dem Dataframe nur die gewünschten Daten gefiltert werden! Was passiert, wenn `penguins[penguins["Species"] == "Adelie"]["Culmen Length (mm)"]` eingegeben wird. Was sind die einzelnen Schritte, die hier auf eine Zeile reduziert werden?

Auch dieser Wert sollte euch aus der Tabelle oben bekannt vorkommen. Es ist die within SS. Also der Teil der Gesamtvarianz der nicht durch eine Gruppenzugehörigkeit erklärt werden kann, bzw. der Teil an Varianz der noch innerhalb der Gruppen existiert. 

### $SS_{between}$

Die Varianz zwischen den Gruppen müssen wir jetzt auch noch errechnen. Da sich die Varianz am Ende zur Gesamtvarianz aufsummieren können wir das mit einer einfachen Differenz:

In [ ]:
SS_within = SS_Adelie + SS_Gentoo + SS_Chinstrap

SS_between1 = SS - SS_within

print(
    f"Summe der Varianz zwischen den Gruppen ausgerechnet aus der Gesamtvarianz: {SS_between1:.2f}.\n"
    f"Between Group Varianz aus der ANOVA-Tabelle: {anova_results['SS']['Species']:.2f}."
)

Die andere Möglichkeit die Between Varianz zu errechnen erklärt vielleicht etwas anschaulicher, warum das nun die between Varianz ist. Hierfür wird ein Gesamtmittelwert gebildet und von den Gruppenmittelwerten abgezogen. Diese anschließend quadriert und mit der Gruppengröße multipliziert. Wir berechnen also die Varianz der Gruppenmittelwerte um den Gesamtmittelwert und tun am Ende so, als ob wir alle Individuuen in der Stichprobe durch die Mittelwerte in der Gruppe erklärt werden könnten (multiplizieren mal n).

$$SS_{between} = \sum n_i(M_i - M_{gesamt})^2$$

Damit der Code um das zu berechnen nicht zu unübersichtlich wird, nutzen wir einige bekannte Pandas Funktionen und einige neue. `.groupby´ erlaubt uns unsere Daten nach Kategorien zu gruppieren und anschließend Statistiken darauf anzuwenden. So können wir für jede Spezies den Mittelwert der Schnabellänge herausfinden.

In [ ]:
Group_means = penguins.groupby("Species").mean()["Culmen Length (mm)"]

Group_means

Außerdem benötigen wir den Gesamtmittelwert und die Gruppengrößen. Den Gesamtmittelwert haben wir zum Glück oben bereits einmal ausgerechnet und `M_gesamt` genannt.

In [ ]:
Group_n = penguins.groupby("Species").count()["Culmen Length (mm)"]
# wir nutzen hier groupby statt value_counts, um die Reihenfolge beizubehalten

Group_n

In [ ]:
SS_between2 = np.sum(Group_n * (Group_means - M_gesamt) ** 2)

print(
    f"Summe der Varianz zwischen den Gruppen ausgerechnet aus der Gesamtvarianz: {SS_between1:.2f}.\n"
    f"Between Group Varianz aus der ANOVA-Tabelle: {anova_results['SS']['Species']:.2f}.\n"
    f"Between Group Varianz errechnet als Varianz der Gruppenmittelwerte vom Gesamtmittelwert: {SS_between2:.2f}."
)

## Mean Squares und Freiheitsgrade

Um von der SS auf mean square (MS) zu kommen, brauchen wir die sogennanten Freiheitsgrade (engl., *degrees of freedom*, DF). Diese beschreiben, sehr abstrakt um wie viele Dimensionen wir unsere Daten reduzieren, um sie zu beschreiben. Die Berechnung der mean squares ist eine Standardisierung und erleichtert uns damit die Interpretation.

Was das bedeutet, wird hoffentlich anhand des Beispiels klarer.

Die Formel für das $MS_{between}$ ist:

$$MS_{between} = \frac{SS_{between}}{k - 1} = \frac{SS_{between}}{df_{between}}$$ 


k steht für die Anzahl der Gruppen. Wenn ihr an die zweite Möglichkeit denkt die between Varianz zu berechnen, haben wir dort die die Varianz der 3 Gruppenmittelwerte um den einzelnen Gesamtmittelwert berechnet. Die drei Gruppenmittelwerte geben also hier unsere Anzahl an Dimensionen an, in der unsere "Stichprobe" variieren kann. Der Gesamtmittelwert wäre quasi könnte man als deskriptive Beschreibung der Gruppenmittelwerte nutzen, diese deskriptive Beschreibung nutzt allerdings eine Dimension unserer k = 3 Dimensionen. Das bedeutet dass "- 1" im Nenner des Bruches durch diesen Mittelwert zustande kommt. 

In [ ]:
k = penguins["Species"].nunique()  # Anzahl an einzigartigen Einträgen in der Spalte.

MS_between = SS_between2 / (k - 1)

print(
    f"Mean square between berechnet: {MS_between:.2f}\n"
    f"MS between aus der ANOVA-Tabelle: {anova_results['MS']['Species']:.2f}"
)

Auch den Wert kennen wir schon aus der ANOVA-Tabelle von oben.

Bei der Berechnung der within Varianz hingegen haben wir oben für jede Gruppe an Pinguinen einen Mittelwert gebildet. In unserem Fall bedeutet das: $df_{within} = (N_{Adelie} - 1) + (N_{Gentoo} - 1) + (N_{Chinstrap} -1)$

Allgemein:

$$MS_{within} = \frac{SS_{within}}{N - k} = \frac{SS_{within}}{df_{within}}$$

wobei k wieder die Anzahl der Gruppen ist


In [ ]:
N = len(penguins)

MS_within = SS_within / (N - k)

print(
    f"Mean square within berechnet: {MS_within:.2f}\n"
    f"MS within aus der ANOVA-Tabelle: {anova_results['MS']['Within']:.2f}"
)

## F-Wert

Der F-Wert ist letztendlich nur die Relation der $MS_{between}$ und $MS_{within}$ und gibt uns damit einen Anhaltspunkt wie viel der Gesamtvarianz wir erklären können:

$$F = \frac{MS_{between}}{MS_{within}}$$

In [ ]:
F = MS_between / MS_within

print(
    f"F-Wert berechnet: {F:.2f}\n"
    f"F-Wert der ANOVA-Tabelle: {anova_results['F']['Species']:.2f}"
)

Mithilfe des F-Werts und den Freiheitsgraden kann man anschließend den p-Wert aus einer Verteilungstabelle ablesen. Wichtig ist, dass die F-Verteilung eine Wahrscheinlichkeitsdichtefunktion ist. Die genaue Mathematik dahinter übersteigt auch mein Wissen.

Hier aber eine gute <a href="https://www.statlect.com/probability-distributions/F-distribution#DensityPlots">Ressource</a>, an der man sieht, wie die Freiheitsgrade auf die F-Verteilung wirken.

## (Partielles) $\eta^2$

Zuletzt noch die Effektstärke, die bei einer ANOVA angegeben werden kann. Eta Quadrat oder das partielle Eta Quadrat. Die beiden Werte sind gleich, wenn man nur eine unabhängige Variable hat. Daher auch in unserem Beispiel. Trotzdem hier einmal beide Formeln.

$$\eta^2 = \frac{SS_{effect}}{SS_{gesamt}}$$

und 

$$partielles\:\eta^2 = \frac{SS_{effect}}{SS_{effect} + SS_{within}}$$

Da wir nur eine unabhängige Variable haben, entspricht unsere $SS_{effect}$ immer unserer $SS_{between}$. Ihr könnt euch aber vielleicht vorstellen, dass die beiden Formeln unterschiedliche Werte produzieren, wenn man mehrere Effekte hat.

In [ ]:
eta = SS_between2 / SS

p_eta = SS_between2 / (SS_between2 + SS_within)

print(f"{eta:.2f} | {p_eta:.2f} | {anova_results['np2']['Species']:.2f}")

Q: Kannst du genauso für "Culmen Depth (mm)" eine ANOVA rechnen und im Anschluss die Tabelle reproduzieren? Versuche dir hierbei nochmal klarzumachen, welche Varianzen verglichen werden und in welchen Beziehungen die einzelnen Outputs der ANOVA-Tabelle stehen!

## Quellen

1. http://pytolearn.csd.auth.gr/d1-hyptest/12/ptl_anova1.html
2. http://vassarstats.net/textbook/index.html
